# 🚀 HRHUB - Complete Bilateral Matching System

## 🎯 System Architecture:

```
Candidates (9.5K) ←→ Postings (700) ←→ Companies (180K)
         ↓                ↓                  ↓
    Skills text    Job requirements    Enriched profiles
         ↓                ↓                  ↓
    Embeddings ←―――――― SAME SPACE ℝ³⁸⁴ ―――――→
```

## 🔑 Key Innovation:

**Use postings to enrich company profiles** so they speak the same language as candidates!

- Companies describe: "We are in tech industry"
- Postings translate: "We need Python, AWS, React"
- Result: Companies can match with candidates!

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 📦 Step 1: Install & Import

Roger, what means pandas?

In [ ]:
!pip install -q sentence-transformers plotly anthropic scikit-learn umap-learn

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages ready!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 13.0 MB/s eta 0:00:00
✅ All packages ready!


## 📂 Step 2: Load ALL Datasets

In [ ]:
print("📂 Loading all datasets...\n")
print("=" * 70)

# Load candidates
candidates = pd.read_csv('resume_data.csv')
print(f"✅ Candidates: {len(candidates):,} rows × {len(candidates.columns)} columns")

# Load companies base
companies_base = pd.read_csv('companies.csv')
print(f"✅ Companies (base): {len(companies_base):,} rows")

# Load company enrichment data
company_industries = pd.read_csv('company_industries.csv')
print(f"✅ Company industries: {len(company_industries):,} rows")

company_specialties = pd.read_csv('company_specialities.csv') # Corrected filename
print(f"✅ Company specialties: {len(company_specialties):,} rows")

employee_counts = pd.read_csv('employee_counts.csv')
print(f"✅ Employee counts: {len(employee_counts):,} rows")

# Load POSTINGS (THE BRIDGE!)
postings = pd.read_csv('postings.csv', on_bad_lines='skip', engine='python')
print(f"✅ Postings: {len(postings):,} rows × {len(postings.columns)} columns")

# Load job-related tables
try:
    job_skills = pd.read_csv('job_skills.csv')
    print(f"✅ Job skills: {len(job_skills):,} rows")
except:
    job_skills = None
    print("⚠️  Job skills not found (optional)")

try:
    job_industries = pd.read_csv('job_industries.csv')
    print(f"✅ Job industries: {len(job_industries):,} rows")
except:
    job_industries = None
    print("⚠️  Job industries not found (optional)")

print("\n" + "=" * 70)
print("✅ All datasets loaded!\n")

📂 Loading all datasets...

✅ Candidates: 9,544 rows × 35 columns
✅ Companies (base): 24,473 rows
✅ Company industries: 24,375 rows
✅ Company specialties: 169,387 rows
✅ Employee counts: 35,787 rows
✅ Postings: 97,164 rows × 31 columns
✅ Job skills: 213,768 rows
✅ Job industries: 164,808 rows

✅ All datasets loaded!



In [ ]:
candidates.head()

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000


## 🔗 Step 3: Merge Company Data

In [ ]:
print("🔗 Merging company data...\n")

# Aggregate industries
company_industries_agg = company_industries.groupby('company_id')['industry'].apply(
    lambda x: ', '.join(map(str, x.tolist()))
).reset_index()
company_industries_agg.columns = ['company_id', 'industries_list']
print(f"✅ Aggregated industries for {len(company_industries_agg):,} companies")

# Aggregate specialties
company_specialties_agg = company_specialties.groupby('company_id')['speciality'].apply(
    lambda x: ' | '.join(x.astype(str).tolist())
).reset_index()
company_specialties_agg.columns = ['company_id', 'specialties_list']
print(f"✅ Aggregated specialties for {len(company_specialties_agg):,} companies")

# Start with base
companies_merged = companies_base.copy()

# Merge industries
companies_merged = companies_merged.merge(
    company_industries_agg,
    on='company_id',
    how='left'
)

# Merge specialties
companies_merged = companies_merged.merge(
    company_specialties_agg,
    on='company_id',
    how='left'
)

# Merge employee counts
companies_merged = companies_merged.merge(
    employee_counts,
    on='company_id',
    how='left'
)

print(f"\n✅ Base company merge complete: {len(companies_merged):,} companies")
print(f"📊 Columns: {companies_merged.columns.tolist()[:10]}...\n")

🔗 Merging company data...

✅ Aggregated industries for 24,365 companies
✅ Aggregated specialties for 17,780 companies

✅ Base company merge complete: 35,787 companies
📊 Columns: ['company_id', 'name', 'description', 'company_size', 'state', 'country', 'city', 'zip_code', 'address', 'url']...



## 🌉 Step 4: Enrich Companies with Postings (THE BRIDGE!)

**This is the key step!** Postings tell us what companies actually need.

In [ ]:
print("🌉 Enriching companies with job posting data...\n")
print("=" * 70)
print("KEY INSIGHT: Postings contain the 'requirements language'")
print("that bridges companies and candidates!")
print("=" * 70 + "\n")

# Clean postings
postings = postings.fillna('')

# Ensure company_id in postings is of consistent type (e.g., string) before grouping
postings['company_id'] = postings['company_id'].astype(str)

# Aggregate postings per company
postings_agg = postings.groupby('company_id').agg({
    'title': lambda x: ' | '.join(x.astype(str).tolist()[:10]),  # Top 10 job titles
    'description': lambda x: ' '.join(x.astype(str).tolist()[:5]),  # Top 5 descriptions (truncated)
    'skills_desc': lambda x: ' | '.join(x.dropna().astype(str).tolist()),  # All skills
    'formatted_experience_level': lambda x: ' | '.join(x.dropna().unique().astype(str)),
    'formatted_work_type': lambda x: ' | '.join(x.dropna().unique().astype(str))
}).reset_index()

postings_agg.columns = [
    'company_id',
    'posted_job_titles',
    'posted_descriptions',
    'required_skills',
    'experience_levels',
    'work_types'
]

print(f"✅ Aggregated postings for {len(postings_agg):,} companies")
print(f"\n💡 These {len(postings_agg):,} companies have explicit requirements!\n")

# Ensure company_id in companies_merged is of consistent type (e.g., string) before merging
companies_merged['company_id'] = companies_merged['company_id'].astype(str)

# Merge postings into companies
companies_full = companies_merged.merge(
    postings_agg,
    on='company_id',
    how='left'
)

# Fill NaN
companies_full = companies_full.fillna('')

print(f"✅ ENRICHED COMPANIES CREATED!")
print(f"📊 Final: {len(companies_full):,} companies × {len(companies_full.columns)} columns")
print(f"\n📋 New columns from postings:")
print(f"   - posted_job_titles")
print(f"   - posted_descriptions")
print(f"   - required_skills ← KEY FOR MATCHING!")
print(f"   - experience_levels")
print(f"   - work_types\n")

# Show sample
print("👀 Sample enriched company:")
companies_with_skills = companies_full[companies_full['required_skills'] != '']

if not companies_with_skills.empty:
    sample_with_postings = companies_with_skills.iloc[0]
    print(f"\nCompany: {sample_with_postings.get('name', 'N/A')}")
    print(f"Industries: {str(sample_with_postings.get('industries_list', ''))[:100]}...")
    print(f"Required Skills: {str(sample_with_postings.get('required_skills', ''))[:100]}...")
    print(f"Job Titles Posted: {str(sample_with_postings.get('posted_job_titles', ''))[:100]}...")
else:
    print("\n⚠️ No companies found with non-empty 'required_skills' after enrichment.")
    print("   Showing a sample of a generic enriched company instead (may have empty skills).")
    sample_with_postings = companies_full.iloc[0] # Fallback to any company
    print(f"\nCompany: {sample_with_postings.get('name', 'N/A')}")
    print(f"Industries: {str(sample_with_postings.get('industries_list', ''))[:100]}...")
    print(f"Required Skills: {str(sample_with_postings.get('required_skills', ''))[:100]}...")
    print(f"Job Titles Posted: {str(sample_with_postings.get('posted_job_titles', ''))[:100]}...")

🌉 Enriching companies with job posting data...

KEY INSIGHT: Postings contain the 'requirements language'
that bridges companies and candidates!

✅ Aggregated postings for 15,097 companies

💡 These 15,097 companies have explicit requirements!

✅ ENRICHED COMPANIES CREATED!
📊 Final: 35,787 companies × 20 columns

📋 New columns from postings:
   - posted_job_titles
   - posted_descriptions
   - required_skills ← KEY FOR MATCHING!
   - experience_levels
   - work_types

👀 Sample enriched company:

⚠️ No companies found with non-empty 'required_skills' after enrichment.
   Showing a sample of a generic enriched company instead (may have empty skills).

Company: IBM
Industries: IT Services and IT Consulting...
Required Skills: ...
Job Titles Posted: ...


## 📂 Step 5: Load & Clean Candidates

In [ ]:
# Clean candidates
candidates = candidates.fillna('')

print(f"✅ Candidates cleaned: {len(candidates):,} rows")
print(f"📋 Columns: {candidates.columns.tolist()[:10]}...")
candidates.head(3)

✅ Candidates cleaned: 9,544 rows
📋 Columns: ['address', 'career_objective', 'skills', 'educational_institution_name', 'degree_names', 'passing_years', 'educational_results', 'result_types', 'major_field_of_studies', 'professional_company_names']...


,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,,,,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,,Technical Support\nTroubleshooting\nCollaborat...,,0.850000
1,,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,,,,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),,Machine Learning Leadership\nCross-Functional ...,,0.750000
2,,,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,,,,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667


## 📝 Step 6: Create Aligned Text Representations

**CRITICAL:** Both entities must use the same vocabulary!

In [ ]:
print("📝 Creating ALIGNED text representations...\n")
print("=" * 70)
print("ALIGNMENT STRATEGY:")
print("• Candidates: Describe skills, experience, education")
print("• Companies: Describe what they NEED (from postings!)")
print("• Result: Both use 'skills language' → same vector space!")
print("=" * 70 + "\n")

# ========================================================================
# CANDIDATE TEXT - Professional offering
# ========================================================================
def make_candidate_text(row):
    """
    Candidate text focuses on:
    - What skills I have
    - What experience I bring
    - What value I offer
    """
    parts = []

    # Professional identity
    if row.get('career_objective'):
        parts.append(f"Professional seeking: {row['career_objective']}")

    if row.get('job_position_name'):
        parts.append(f"Target role: {row['job_position_name']}")

    # SKILLS (most important for matching!)
    all_skills = []
    if row.get('skills'):
        all_skills.append(row['skills'])
    if row.get('related_skills_in_job'):
        all_skills.append(row['related_skills_in_job'])
    if row.get('certification_skills'):
        all_skills.append(row['certification_skills'])
    if row.get('skills_required'):  # Skills they're looking for in jobs
        all_skills.append(row['skills_required'])

    if all_skills:
        parts.append(f"Skills and expertise: {' | '.join(all_skills)}")

    # EXPERIENCE
    if row.get('positions'):
        parts.append(f"Experience in roles: {row['positions']}")

    if row.get('professional_company_names'):
        parts.append(f"Companies worked at: {row['professional_company_names']}")

    if row.get('responsibilities'):
        resp = str(row['responsibilities'])[:250]
        parts.append(f"Responsibilities: {resp}")

    # EDUCATION
    edu_parts = []
    if row.get('degree_names'):
        edu_parts.append(row['degree_names'])
    if row.get('major_field_of_studies'):
        edu_parts.append(f"in {row['major_field_of_studies']}")
    if row.get('educational_institution_name'):
        edu_parts.append(f"from {row['educational_institution_name']}")

    if edu_parts:
        parts.append(f"Education: {' '.join(edu_parts)}")

    # ADDITIONAL
    if row.get('languages'):
        parts.append(f"Languages: {row['languages']}")

    if row.get('certification_providers'):
        parts.append(f"Certifications from: {row['certification_providers']}")

    if row.get('extra_curricular_activity_types'):
        parts.append(f"Activities: {row['extra_curricular_activity_types']}")

    return ' || '.join(parts) if parts else "Professional profile"


# ========================================================================
# COMPANY TEXT - Job requirements (enriched with postings!)
# ========================================================================
def make_company_text(row):
    """
    Company text focuses on:
    - What skills we need (from postings!)
    - What roles we're hiring for
    - What our company does
    """
    parts = []

    # Company identity
    if row.get('name'):
        parts.append(f"Company: {row['name']}")

    # REQUIRED SKILLS (from postings - KEY!)
    if row.get('required_skills'):
        parts.append(f"Looking for skills: {row['required_skills']}")

    # JOB TITLES (from postings)
    if row.get('posted_job_titles'):
        parts.append(f"Hiring for roles: {row['posted_job_titles']}")

    # EXPERIENCE LEVELS (from postings)
    if row.get('experience_levels'):
        parts.append(f"Experience levels: {row['experience_levels']}")

    # Industries & specialties
    if row.get('industries_list'):
        parts.append(f"Industries: {row['industries_list']}")

    if row.get('specialties_list'):
        parts.append(f"Specialties: {row['specialties_list']}")

    # Company description
    if row.get('description'):
        desc = str(row['description'])[:300]
        parts.append(f"About: {desc}")

    # Posted descriptions (gives context)
    if row.get('posted_descriptions'):
        posted_desc = str(row['posted_descriptions'])[:200]
        parts.append(f"Job descriptions: {posted_desc}")

    # Company size
    if row.get('employee_count'):
        parts.append(f"Company size: {row['employee_count']} employees")

    # Location
    loc = []
    if row.get('city'): loc.append(row['city'])
    if row.get('state'): loc.append(row['state'])
    if row.get('country'): loc.append(row['country'])
    if loc:
        parts.append(f"Location: {', '.join(loc)}")

    # Work types
    if row.get('work_types'):
        parts.append(f"Work arrangement: {row['work_types']}")

    return ' || '.join(parts) if parts else "Company profile"


# ========================================================================
# APPLY TO DATAFRAMES
# ========================================================================
print("🔄 Generating candidate texts...")
candidates['text'] = candidates.apply(make_candidate_text, axis=1)

print("🔄 Generating company texts...")
companies_full['text'] = companies_full.apply(make_company_text, axis=1)

print("\n✅ ALIGNED texts created!\n")

# Compare vocabularies
print("=" * 70)
print("CANDIDATE SAMPLE:")
print(candidates['text'].iloc[0][:500])
print("\n" + "=" * 70)
print("COMPANY SAMPLE (with postings data):")
# Find company with postings
companies_with_skills_text = companies_full[companies_full['required_skills'] != '']
if not companies_with_skills_text.empty:
    company_with_postings = companies_with_skills_text.iloc[0]
    print(company_with_postings['text'][:500])
else:
    print("\n⚠️ No companies found with non-empty 'required_skills' in their text representation.")
    print("   Showing a sample of a generic enriched company instead (may have empty skills).")
    sample_company = companies_full.iloc[0] # Fallback to any company
    print(sample_company['text'][:500])

print("=" * 70)

print("\n💡 Notice: Both now use SKILLS LANGUAGE!")
print("   Candidate: 'Skills and expertise: Python, Java'")
print("   Company: 'Looking for skills: Python, AWS'")
print("   → They can now be compared in the same space!\n")

📝 Creating ALIGNED text representations...

ALIGNMENT STRATEGY:
• Candidates: Describe skills, experience, education
• Companies: Describe what they NEED (from postings!)
• Result: Both use 'skills language' → same vector space!

🔄 Generating candidate texts...
🔄 Generating company texts...

✅ ALIGNED texts created!

CANDIDATE SAMPLE:
Professional seeking: Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development. || Skills and expertise: ['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'I

COMPANY SAMPLE (with postings data):

⚠️ No companies found with non-empty 'required_skills' in their text representation.
   Showing a sample of a generic enric

## 🧠 Step 7: Generate Embeddings (ℝ³⁸⁴)

Transform aligned text → vectors in same mathematical space

In [ ]:
print("🧠 Loading embedding model...\n")
model = SentenceTransformer('all-MiniLM-L6-v2')

embedding_dim = model.get_sentence_embedding_dimension()
print(f"✅ Model loaded! Embedding dimension: ℝ^{embedding_dim}\n")

In [ ]:
# Save the embedding_dim
print("\n💾 Saving vectors for future use...")
np.save('embedding_dim.npy', embedding_dim)
print("✅ Vectors saved!")

In [ ]:
print("🔄 Generating candidate vectors...")
print(f"   ({len(candidates):,} candidates × ~2-3 minutes)\n")
cand_vectors = model.encode(
    candidates['text'].tolist(),
    show_progress_bar=True,
    batch_size=32
)

print("\n🔄 Generating company vectors...")
print(f"   ({len(companies_full):,} companies × ~15-20 minutes)\n")
comp_vectors = model.encode(
    companies_full['text'].tolist(),
    show_progress_bar=True,
    batch_size=64
)

print("\n" + "=" * 70)
print("✅ VECTORS CREATED IN SAME SPACE!")
print("=" * 70)
print(f"📊 Candidate vectors: {cand_vectors.shape}")
print(f"📊 Company vectors: {comp_vectors.shape}")
print(f"\n🎯 Both live in ℝ^{embedding_dim}!")
print(f"🎯 Now companies with 'Python' requirements will be")
print(f"   CLOSE to candidates with 'Python' skills!\n")

🧠 Loading embedding model...



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ Model loaded! Embedding dimension: ℝ^384

🔄 Generating candidate vectors...
   (9,544 candidates × ~2-3 minutes)



Batches:   0%|          | 0/299 [00:00<?, ?it/s]


🔄 Generating company vectors...
   (35,787 companies × ~15-20 minutes)



Batches:   0%|          | 0/560 [00:00<?, ?it/s]


✅ VECTORS CREATED IN SAME SPACE!
📊 Candidate vectors: (9544, 384)
📊 Company vectors: (35787, 384)

🎯 Both live in ℝ^384!
🎯 Now companies with 'Python' requirements will be
   CLOSE to candidates with 'Python' skills!



In [ ]:
# Save for next time
print("\n💾 Saving vectors for future use...")
np.save('cand_vectors.npy', cand_vectors)
np.save('comp_vectors.npy', comp_vectors)
print("✅ Vectors saved!")


💾 Saving vectors for future use...
✅ Vectors saved!


In [ ]:
print("\n📦 Loading cached vectors...")
cand_vectors = np.load('cand_vectors.npy')
comp_vectors = np.load('comp_vectors.npy')
print("✅ Loaded from cache!")


📦 Loading cached vectors...
✅ Loaded from cache!


## 🎯 Step 8: Matching Engine

In [ ]:
def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def find_top_matches(candidate_idx, top_k=10):
    """
    Find top K company matches for a candidate.

    Returns: List of (company_idx, similarity_score)
    """
    cand_vec = cand_vectors[candidate_idx]

    scores = []
    for i, comp_vec in enumerate(comp_vectors):
        score = cosine_similarity(cand_vec, comp_vec)
        scores.append((i, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

print("✅ Matching engine ready!")
print(f"📊 Can match {len(candidates):,} candidates with {len(companies_full):,} companies\n")

✅ Matching engine ready!
📊 Can match 9,544 candidates with 35,787 companies



## 🔍 Step 9: Test Matching

In [ ]:
print("🔍 Finding top 10 matches for Candidate #0...\n")

matches = find_top_matches(0, top_k=10)

print("🎯 Top 10 Company Matches:\n")
print("=" * 90)
print(f"{'Rank':<6} {'Score':<8} {'Company':<35} {'Skills Needed':<40}")
print("=" * 90)

for rank, (comp_idx, score) in enumerate(matches, 1):
    company = companies_full.iloc[comp_idx]
    name = company.get('name', 'N/A')[:33]
    skills = company.get('required_skills', 'N/A')[:38]
    print(f"{rank:<6} {score:.4f}   {name:<35} {skills}")

print("=" * 90)

print("\n💡 If scores are good (>0.5), the alignment worked!")
print("   High scores = Company needs match candidate skills\n")

🔍 Finding top 10 matches for Candidate #0...

🎯 Top 10 Company Matches:

Rank   Score    Company                             Skills Needed                           
1      0.7028   Anblicks                            
2      0.7026   iO Associates - US                  
3      0.7026   Anblicks                            
4      0.7019   iO Associates - US                  
5      0.7010   Anblicks                            
6      0.6849   DATAECONOMY                         
7      0.6827   Datavail                            
8      0.6800   DATAECONOMY                         
9      0.6784   DATAECONOMY                         
10     0.6776   BitPusher                           

💡 If scores are good (>0.5), the alignment worked!
   High scores = Company needs match candidate skills



## 📊 Step 10: Visualize Vector Space

See where candidates and companies live in ℝ³⁸⁴ (projected to ℝ²)

In [ ]:
print("🎨 VECTOR SPACE VISUALIZATION\n")
print("=" * 70)

# Sample for visualization
n_cand_viz = min(500, len(candidates))
n_comp_viz = min(2000, len(companies_full))

print(f"📊 Visualizing:")
print(f"   • {n_cand_viz} candidates")
print(f"   • {n_comp_viz} companies")
print(f"   • From ℝ^{embedding_dim} → ℝ² (t-SNE projection)\n")

# Sample vectors
cand_sample = cand_vectors[:n_cand_viz]
comp_sample = comp_vectors[:n_comp_viz]

# Combine
all_vectors = np.vstack([cand_sample, comp_sample])

print("🔄 Running t-SNE (2-3 minutes)...")
tsne = TSNE(
    n_components=2,
    perplexity=30,
    random_state=42,
    n_iter=1000,
    verbose=1
)

vectors_2d = tsne.fit_transform(all_vectors)

# Split
cand_2d = vectors_2d[:n_cand_viz]
comp_2d = vectors_2d[n_cand_viz:]

print("\n✅ t-SNE complete!\n")

🎨 VECTOR SPACE VISUALIZATION

📊 Visualizing:
   • 500 candidates
   • 2000 companies


NameError: name 'embedding_dim' is not defined

In [ ]:
# Create plot
fig = go.Figure()

# Companies (red)
fig.add_trace(go.Scatter(
    x=comp_2d[:, 0],
    y=comp_2d[:, 1],
    mode='markers',
    name='Companies',
    marker=dict(
        size=6,
        color='#ff6b6b',
        opacity=0.6
    ),
    text=[f"Company {i}: {companies_full.iloc[i].get('name', 'N/A')[:30]}"
          for i in range(n_comp_viz)],
    hovertemplate='<b>%{text}</b><extra></extra>'
))

# Candidates (green)
fig.add_trace(go.Scatter(
    x=cand_2d[:, 0],
    y=cand_2d[:, 1],
    mode='markers',
    name='Candidates',
    marker=dict(
        size=10,
        color='#00ff00',
        opacity=0.8,
        line=dict(width=1, color='white')
    ),
    text=[f"Candidate {i}" for i in range(n_cand_viz)],
    hovertemplate='<b>%{text}</b><extra></extra>'
))

fig.update_layout(
    title='Vector Space: Candidates & Companies (with Postings Enrichment)',
    xaxis_title='Dimension 1',
    yaxis_title='Dimension 2',
    width=1200,
    height=800,
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#0d0d0d',
    font=dict(color='white')
)

fig.show()

print("✅ Visualization complete!\n")
print("💡 KEY OBSERVATIONS:")
print("   • Green = Candidates | Red = Companies")
print("   • If they OVERLAP → Good! Alignment worked!")
print("   • If still separated → Need more postings data")
print("   • Clusters = Similar skill profiles grouped\n")

✅ Visualization complete!

💡 KEY OBSERVATIONS:
   • Green = Candidates | Red = Companies
   • If they OVERLAP → Good! Alignment worked!
   • If still separated → Need more postings data
   • Clusters = Similar skill profiles grouped



## 🔍 Step 11: Highlight Specific Candidate + Matches

In [ ]:
target_candidate = 0

print(f"🔍 Analyzing Candidate #{target_candidate}...\n")

matches = find_top_matches(target_candidate, top_k=10)
match_indices = [comp_idx for comp_idx, score in matches if comp_idx < n_comp_viz]

# Create highlighted plot
fig2 = go.Figure()

# All companies (background)
fig2.add_trace(go.Scatter(
    x=comp_2d[:, 0],
    y=comp_2d[:, 1],
    mode='markers',
    name='All Companies',
    marker=dict(size=4, color='#ff6b6b', opacity=0.3),
    showlegend=True
))

# Top matches (highlighted)
if match_indices:
    match_positions = comp_2d[match_indices]
    fig2.add_trace(go.Scatter(
        x=match_positions[:, 0],
        y=match_positions[:, 1],
        mode='markers',
        name='Top Matches',
        marker=dict(
            size=15,
            color='#ff0000',
            line=dict(width=2, color='white')
        ),
        text=[f"Match #{i+1}: {companies_full.iloc[match_indices[i]].get('name', 'N/A')[:30]}<br>Score: {matches[i][1]:.3f}"
              for i in range(len(match_indices))],
        hovertemplate='<b>%{text}</b><extra></extra>'
    ))

# Target candidate
fig2.add_trace(go.Scatter(
    x=[cand_2d[target_candidate, 0]],
    y=[cand_2d[target_candidate, 1]],
    mode='markers',
    name=f'Candidate #{target_candidate}',
    marker=dict(
        size=25,
        color='#00ff00',
        symbol='star',
        line=dict(width=3, color='white')
    )
))

# Connection lines
for i, match_idx in enumerate(match_indices[:5]):
    fig2.add_trace(go.Scatter(
        x=[cand_2d[target_candidate, 0], comp_2d[match_idx, 0]],
        y=[cand_2d[target_candidate, 1], comp_2d[match_idx, 1]],
        mode='lines',
        line=dict(color='yellow', width=1, dash='dot'),
        opacity=0.5,
        showlegend=False
    ))

fig2.update_layout(
    title=f'Candidate #{target_candidate} and Top Matches',
    xaxis_title='Dimension 1',
    yaxis_title='Dimension 2',
    width=1200,
    height=800,
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#0d0d0d',
    font=dict(color='white')
)

fig2.show()

print("✅ Highlighted visualization created!")
print(f"   ⭐ Green star = Candidate #{target_candidate}")
print(f"   🔴 Red dots = Top matches")
print(f"   💛 Yellow lines = Connections in vector space\n")

🔍 Analyzing Candidate #0...



✅ Highlighted visualization created!
   ⭐ Green star = Candidate #0
   🔴 Red dots = Top matches
   💛 Yellow lines = Connections in vector space



## 💾 Step 12: Export Results

In [ ]:
# Generate matches for sample
results = []
export_sample = min(500, len(candidates))

print(f"💾 Generating matches for {export_sample} candidates...\n")

for i in range(export_sample):
    if i % 50 == 0:
        print(f"   Progress: {i}/{export_sample}")

    matches = find_top_matches(i, top_k=10)

    for rank, (comp_idx, score) in enumerate(matches, 1):
        company = companies_full.iloc[comp_idx]
        results.append({
            'candidate_id': i,
            'company_id': company.get('company_id'),
            'company_name': company.get('name', 'N/A'),
            'rank': rank,
            'similarity_score': float(score),
            'required_skills': company.get('required_skills', 'N/A')[:100],
            'posted_jobs': company.get('posted_job_titles', 'N/A')[:100]
        })

results_df = pd.DataFrame(results)
results_df.to_csv('hrhub_matches_with_postings.csv', index=False)

print(f"\n✅ Exported {len(results_df):,} matches!")
print(f"📄 File: hrhub_matches_with_postings.csv\n")
results_df.head(20)

## 🎉 COMPLETE!

### ✅ What you have:

1. **Enriched companies** with job posting data (requirements, skills needed)
2. **Aligned text representations** (both use "skills language")
3. **Vectors in same space** ℝ³⁸⁴
4. **Cosine similarity matching**
5. **Vector space visualization**
6. **Exported results**

### 🚀 Next steps:

1. **Train LLM on patterns:** "Company in industry X historically needs skills Y"
2. **Predict for companies without postings:** Use learned patterns
3. **Add weights:** Let users tune dimension importance
4. **Build UI:** Interactive matching interface
5. **LLM explanations:** Why these matches make sense

### 💡 Key insight achieved:

**Postings bridge the gap!** They translate "what companies are" into "what companies need" - the same language candidates speak!

---

In [ ]:
# ============================================================================
# NETWORK GRAPH WITH NODES & EDGES (usando networkx + plotly)
# ============================================================================

import networkx as nx
import plotly.graph_objects as go

print("🕸️  Creating NETWORK GRAPH with nodes and edges...\n")

# ----------------------------------------------------------------------------
# STEP 1: Create NetworkX graph
# ----------------------------------------------------------------------------

G = nx.Graph()

# Sample for visualization
n_cand_sample = min(20, len(candidates))  # 20 candidates
top_k_per_cand = 5  # Top 5 matches each

print(f"📊 Building network:")
print(f"   • {n_cand_sample} candidate nodes")
print(f"   • {top_k_per_cand} companies per candidate")

# Add candidate nodes
for i in range(n_cand_sample):
    G.add_node(
        f"C{i}",
        node_type='candidate',
        label=f"Candidate {i}"
    )

# Add company nodes and edges
companies_in_graph = set()

for i in range(n_cand_sample):
    matches = find_top_matches(i, top_k=top_k_per_cand)

    for comp_idx, score in matches:
        comp_id = f"J{comp_idx}"

        # Add company node if not exists
        if comp_id not in companies_in_graph:
            company_name = companies_full.iloc[comp_idx].get('name', 'N/A')[:30]
            G.add_node(
                comp_id,
                node_type='company',
                label=company_name
            )
            companies_in_graph.add(comp_id)

        # Add edge (connection!)
        G.add_edge(
            f"C{i}",
            comp_id,
            weight=float(score)
        )

print(f"\n✅ Network created!")
print(f"   • Nodes: {G.number_of_nodes()}")
print(f"   • Edges: {G.number_of_edges()}")

# ----------------------------------------------------------------------------
# STEP 2: Calculate layout (positions using spring layout)
# ----------------------------------------------------------------------------

print(f"\n🔄 Calculating network layout...")

# Use spring layout (force-directed)
pos = nx.spring_layout(
    G,
    k=2,  # Optimal distance between nodes
    iterations=50,
    seed=42
)

print(f"✅ Layout calculated!\n")

# ----------------------------------------------------------------------------
# STEP 3: Create Plotly visualization with nodes and edges
# ----------------------------------------------------------------------------

# Prepare edge traces
edge_trace = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    weight = edge[2]['weight']

    edge_trace.append(
        go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(
                width=weight * 3,  # Thicker = higher similarity
                color='rgba(255, 255, 255, 0.3)'
            ),
            hoverinfo='none',
            showlegend=False
        )
    )

# Prepare node traces (separate for candidates and companies)
candidate_nodes = [n for n, d in G.nodes(data=True) if d['node_type'] == 'candidate']
company_nodes = [n for n, d in G.nodes(data=True) if d['node_type'] == 'company']

# Candidate nodes
cand_x = [pos[node][0] for node in candidate_nodes]
cand_y = [pos[node][1] for node in candidate_nodes]
cand_labels = [G.nodes[node]['label'] for node in candidate_nodes]

candidate_trace = go.Scatter(
    x=cand_x,
    y=cand_y,
    mode='markers+text',
    name='Candidates',
    marker=dict(
        size=25,
        color='#00ff00',
        line=dict(width=2, color='white'),
        symbol='circle'
    ),
    text=cand_labels,
    textposition='top center',
    textfont=dict(size=10, color='white'),
    hovertemplate='<b>%{text}</b><extra></extra>'
)

# Company nodes
comp_x = [pos[node][0] for node in company_nodes]
comp_y = [pos[node][1] for node in company_nodes]
comp_labels = [G.nodes[node]['label'] for node in company_nodes]

company_trace = go.Scatter(
    x=comp_x,
    y=comp_y,
    mode='markers+text',
    name='Companies',
    marker=dict(
        size=15,
        color='#ff6b6b',
        line=dict(width=1, color='white'),
        symbol='square'
    ),
    text=comp_labels,
    textposition='top center',
    textfont=dict(size=8, color='white'),
    hovertemplate='<b>%{text}</b><extra></extra>'
)

# ----------------------------------------------------------------------------
# STEP 4: Create figure
# ----------------------------------------------------------------------------

fig = go.Figure(data=edge_trace + [candidate_trace, company_trace])

fig.update_layout(
    title='Network Graph: Candidates ↔ Companies (with Nodes & Edges)',
    showlegend=True,
    hovermode='closest',
    width=1400,
    height=900,
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#0d0d0d',
    font=dict(color='white'),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.show()

print("✅ NETWORK GRAPH WITH NODES CREATED!")
print("\n💡 Legend:")
print("   🟢 Green circles = Candidates")
print("   🔴 Red squares = Companies")
print("   ━━━ Lines = Matching connections")
print("   Thicker lines = Stronger match\n")

🕸️  Creating NETWORK GRAPH with nodes and edges...

📊 Building network:
   • 20 candidate nodes
   • 5 companies per candidate

✅ Network created!
   • Nodes: 73
   • Edges: 100

🔄 Calculating network layout...
✅ Layout calculated!



✅ NETWORK GRAPH WITH NODES CREATED!

💡 Legend:
   🟢 Green circles = Candidates
   🔴 Red squares = Companies
   ━━━ Lines = Matching connections
   Thicker lines = Stronger match



In [ ]:
!pip install pyvis

from pyvis.network import Network

# Create PyVis network
net = Network(height='800px', width='100%', bgcolor='#222', font_color='white')

# Add nodes
for i in range(n_cand_sample):
    net.add_node(f"C{i}", label=f"Cand {i}", color='green', size=25)

# Add companies and edges
for i in range(n_cand_sample):
    matches = find_top_matches(i, top_k=5)
    for comp_idx, score in matches:
        comp_id = f"J{comp_idx}"
        if comp_id not in net.get_nodes():
            company_name = companies_full.iloc[comp_idx].get('name', 'N/A')[:20]
            net.add_node(comp_id, label=company_name, color='red', size=15)

        # Add edge
        net.add_edge(f"C{i}", comp_id, value=float(score)*10, title=f"{score:.3f}")

# Physics
net.set_options("""
{
  "physics": {
    "enabled": true,
    "solver": "forceAtlas2Based"
  }
}
""")

net.save_graph('network_graph.html')
print("✅ Interactive network saved to network_graph.html")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00
✅ Interactive network saved to network_graph.html


In [ ]:
# ============================================================================
# INTERACTIVE NETWORK GRAPH - PyVis in Colab
# ============================================================================

from pyvis.network import Network
from IPython.display import IFrame, display, HTML
import os

print("🕸️  Creating INTERACTIVE network graph...\n")

# ----------------------------------------------------------------------------
# STEP 1: Create PyVis network
# ----------------------------------------------------------------------------

net = Network(
    height='800px',
    width='100%',
    bgcolor='#222222',
    font_color='white',
    notebook=True,  # Important for Colab!
    cdn_resources='remote'  # Use remote CDN
)

# Sample
n_cand_sample = 15  # 15 candidates
top_k_per_cand = 5   # Top 5 matches each

print(f"📊 Building interactive network:")
print(f"   • {n_cand_sample} candidates")
print(f"   • {top_k_per_cand} companies per candidate\n")

companies_added = set()

# ----------------------------------------------------------------------------
# STEP 2: Add nodes and edges
# ----------------------------------------------------------------------------

for i in range(min(n_cand_sample, len(candidates))):
    # Add candidate node (GREEN, LARGER)
    net.add_node(
        f"C{i}",
        label=f"Candidate {i}",
        color='#00ff00',
        size=25,
        shape='dot',
        title=f"<b>Candidate {i}</b><br>Click to see profile"
    )

    # Get top matches
    matches = find_top_matches(i, top_k=top_k_per_cand)

    for rank, (comp_idx, score) in enumerate(matches, 1):
        comp_id = f"J{comp_idx}"

        # Add company node if not exists (RED, SMALLER)
        if comp_id not in companies_added:
            company_name = companies_full.iloc[comp_idx].get('name', 'N/A')[:30]
            required_skills = companies_full.iloc[comp_idx].get('required_skills', 'N/A')[:100]

            net.add_node(
                comp_id,
                label=company_name[:20],  # Short label
                color='#ff6b6b',
                size=15,
                shape='square',
                title=f"<b>{company_name}</b><br>Skills: {required_skills}<br>Match score: {score:.3f}"
            )
            companies_added.add(comp_id)

        # Add edge with weight
        net.add_edge(
            f"C{i}",
            comp_id,
            value=float(score) * 10,  # Thickness
            title=f"Rank #{rank}<br>Similarity: {float(score):.3f}",
            color='rgba(255, 255, 255, 0.5)'
        )

print(f"✅ Network built:")
print(f"   • {len(net.nodes)} nodes")
print(f"   • {len(net.edges)} edges\n")

# ----------------------------------------------------------------------------
# STEP 3: Configure physics for better visualization
# ----------------------------------------------------------------------------

net.set_options("""
var options = {
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 230,
      "springConstant": 0.18,
      "avoidOverlap": 1
    },
    "maxVelocity": 30,
    "solver": "forceAtlas2Based",
    "timestep": 0.35,
    "stabilization": {
      "enabled": true,
      "iterations": 200
    }
  },
  "nodes": {
    "font": {
      "size": 14,
      "color": "white"
    }
  },
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "continuous"
    }
  },
  "interaction": {
    "hover": true,
    "tooltipDelay": 100,
    "navigationButtons": true,
    "keyboard": true
  }
}
""")

# ----------------------------------------------------------------------------
# STEP 4: Save and display in Colab
# ----------------------------------------------------------------------------

html_file = 'interactive_network.html'
net.save_graph(html_file)

print(f"✅ Interactive graph saved to: {html_file}\n")

# Display in Colab using IFrame
print("🎨 Displaying interactive graph in Colab...\n")

# Method 1: IFrame (best for Colab)
display(IFrame(src=html_file, width=1200, height=800))

print("\n" + "="*70)
print("✅ INTERACTIVE GRAPH DISPLAYED!")
print("="*70)
print("\n💡 CONTROLS:")
print("   🖱️  Drag nodes to move them")
print("   🔍 Scroll to zoom in/out")
print("   👆 Click nodes to see details")
print("   ↔️  Drag background to pan")
print("\n🎨 LEGEND:")
print("   🟢 Green dots = Candidates")
print("   🔴 Red squares = Companies")
print("   ━━━ Lines = Match connections")
print("   Thicker lines = Stronger match")
print("\n📥 To download: Right-click graph → Save image")

🕸️  Creating INTERACTIVE network graph...

📊 Building interactive network:
   • 15 candidates
   • 5 companies per candidate

✅ Network built:
   • 64 nodes
   • 75 edges

✅ Interactive graph saved to: interactive_network.html

🎨 Displaying interactive graph in Colab...




✅ INTERACTIVE GRAPH DISPLAYED!

💡 CONTROLS:
   🖱️  Drag nodes to move them
   🔍 Scroll to zoom in/out
   👆 Click nodes to see details
   ↔️  Drag background to pan

🎨 LEGEND:
   🟢 Green dots = Candidates
   🔴 Red squares = Companies
   ━━━ Lines = Match connections
   Thicker lines = Stronger match

📥 To download: Right-click graph → Save image


In [ ]:
# Alternative: Display HTML content directly
print("🔄 Alternative display method...\n")

with open(html_file, 'r', encoding='utf-8') as f:
    html_content = f.read()

# Display HTML directly
display(HTML(html_content))

print("✅ Graph displayed using direct HTML injection!")

🔄 Alternative display method...



✅ Graph displayed using direct HTML injection!


In [ ]:
# ============================================================================
# INTERACTIVE NETWORK GRAPH - PyVis in Colab
# ============================================================================

from pyvis.network import Network
from IPython.display import IFrame, display, HTML
import os

print("🕸️  Creating INTERACTIVE network graph...\n")

# ----------------------------------------------------------------------------
# STEP 1: Create PyVis network
# ----------------------------------------------------------------------------

net = Network(
    height='800px',
    width='100%',
    bgcolor='#222222',
    font_color='white',
    notebook=True,  # Important for Colab!
    cdn_resources='remote'  # Use remote CDN
)

# Sample
n_cand_sample = 100  # 15 candidates
top_k_per_cand = 5   # Top 5 matches each

print(f"📊 Building interactive network:")
print(f"   • {n_cand_sample} candidates")
print(f"   • {top_k_per_cand} companies per candidate\n")

companies_added = set()

# ----------------------------------------------------------------------------
# STEP 2: Add nodes and edges
# ----------------------------------------------------------------------------

for i in range(min(n_cand_sample, len(candidates))):
    # Add candidate node (GREEN, LARGER)
    net.add_node(
        f"C{i}",
        label=f"Candidate {i}",
        color='#00ff00',
        size=25,
        shape='dot',
        title=f"<b>Candidate {i}</b><br>Click to see profile"
    )

    # Get top matches
    matches = find_top_matches(i, top_k=top_k_per_cand)

    for rank, (comp_idx, score) in enumerate(matches, 1):
        comp_id = f"J{comp_idx}"

        # Add company node if not exists (RED, SMALLER)
        if comp_id not in companies_added:
            company_name = companies_full.iloc[comp_idx].get('name', 'N/A')[:30]
            required_skills = companies_full.iloc[comp_idx].get('required_skills', 'N/A')[:100]

            net.add_node(
                comp_id,
                label=company_name[:20],  # Short label
                color='#ff6b6b',
                size=15,
                shape='square',
                title=f"<b>{company_name}</b><br>Skills: {required_skills}<br>Match score: {score:.3f}"
            )
            companies_added.add(comp_id)

        # Add edge with weight
        net.add_edge(
            f"C{i}",
            comp_id,
            value=float(score) * 10,  # Thickness
            title=f"Rank #{rank}<br>Similarity: {float(score):.3f}",
            color='rgba(255, 255, 255, 0.5)'
        )

print(f"✅ Network built:")
print(f"   • {len(net.nodes)} nodes")
print(f"   • {len(net.edges)} edges\n")

# ----------------------------------------------------------------------------
# STEP 3: Configure physics for better visualization
# ----------------------------------------------------------------------------

net.set_options("""
var options = {
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.005,
      "springLength": 230,
      "springConstant": 0.18,
      "avoidOverlap": 1
    },
    "maxVelocity": 30,
    "solver": "forceAtlas2Based",
    "timestep": 0.35,
    "stabilization": {
      "enabled": true,
      "iterations": 200
    }
  },
  "nodes": {
    "font": {
      "size": 14,
      "color": "white"
    }
  },
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "continuous"
    }
  },
  "interaction": {
    "hover": true,
    "tooltipDelay": 100,
    "navigationButtons": true,
    "keyboard": true
  }
}
""")

# ----------------------------------------------------------------------------
# STEP 4: Save and display in Colab
# ----------------------------------------------------------------------------

html_file = 'interactive_network.html'
net.save_graph(html_file)

print(f"✅ Interactive graph saved to: {html_file}\n")

# Display in Colab using IFrame
print("🎨 Displaying interactive graph in Colab...\n")

# Method 1: IFrame (best for Colab)
display(IFrame(src=html_file, width=1200, height=800))

print("\n" + "="*70)
print("✅ INTERACTIVE GRAPH DISPLAYED!")
print("="*70)
print("\n💡 CONTROLS:")
print("   🖱️  Drag nodes to move them")
print("   🔍 Scroll to zoom in/out")
print("   👆 Click nodes to see details")
print("   ↔️  Drag background to pan")
print("\n🎨 LEGEND:")
print("   🟢 Green dots = Candidates")
print("   🔴 Red squares = Companies")
print("   ━━━ Lines = Match connections")
print("   Thicker lines = Stronger match")
print("\n📥 To download: Right-click graph → Save image")

🕸️  Creating INTERACTIVE network graph...

📊 Building interactive network:
   • 100 candidates
   • 5 companies per candidate

✅ Network built:
   • 216 nodes
   • 500 edges

✅ Interactive graph saved to: interactive_network.html

🎨 Displaying interactive graph in Colab...




✅ INTERACTIVE GRAPH DISPLAYED!

💡 CONTROLS:
   🖱️  Drag nodes to move them
   🔍 Scroll to zoom in/out
   👆 Click nodes to see details
   ↔️  Drag background to pan

🎨 LEGEND:
   🟢 Green dots = Candidates
   🔴 Red squares = Companies
   ━━━ Lines = Match connections
   Thicker lines = Stronger match

📥 To download: Right-click graph → Save image


In [ ]:
# Alternative: Display HTML content directly
print("🔄 Alternative display method...\n")

with open(html_file, 'r', encoding='utf-8') as f:
    html_content = f.read()

# Display HTML directly
display(HTML(html_content))

print("✅ Graph displayed using direct HTML injection!")

🔄 Alternative display method...



✅ Graph displayed using direct HTML injection!


In [ ]:
# ============================================================================
# DISPLAY NODE DATA - See what's behind the graph
# ============================================================================

def display_node_data(node_id):
    """
    Print all data for a specific node (candidate or company).

    Args:
        node_id: str like "C76" (candidate) or "J1234" (company)
    """

    print("=" * 80)

    if node_id.startswith('C'):
        # It's a CANDIDATE
        cand_idx = int(node_id[1:])

        if cand_idx >= len(candidates):
            print(f"❌ Candidate {cand_idx} not found!")
            return

        candidate = candidates.iloc[cand_idx]

        print(f"🟢 CANDIDATE #{cand_idx}")
        print("=" * 80)

        # Display all 35 dimensions
        print("\n📊 ALL 35 DIMENSIONS:\n")

        dimensions = [
            ('address', '📍 Address'),
            ('career_objective', '🎯 Career Objective'),
            ('skills', '💻 Skills'),
            ('educational_institution_name', '🎓 Education Institution'),
            ('degree_names', '📜 Degrees'),
            ('passing_years', '📅 Graduation Years'),
            ('educational_results', '⭐ Academic Results'),
            ('result_types', '📊 Result Types'),
            ('major_field_of_studies', '📚 Major Field'),
            ('professional_company_names', '🏢 Companies Worked'),
            ('company_urls', '🔗 Company URLs'),
            ('start_dates', '📅 Start Dates'),
            ('end_dates', '📅 End Dates'),
            ('related_skills_in_job', '🛠️ Job-Related Skills'),
            ('positions', '👔 Positions Held'),
            ('locations', '📍 Work Locations'),
            ('responsibilities', '📋 Responsibilities'),
            ('extra_curricular_activity_types', '🎨 Activities'),
            ('extra_curricular_organization_names', '🏛️ Organizations'),
            ('extra_curricular_organization_links', '🔗 Organization Links'),
            ('role_positions', '👑 Leadership Roles'),
            ('languages', '🌍 Languages'),
            ('proficiency_levels', '📊 Language Proficiency'),
            ('certification_providers', '🏅 Certification Providers'),
            ('certification_skills', '✅ Certified Skills'),
            ('online_links', '🔗 Online Profiles'),
            ('issue_dates', '📅 Certification Issue Dates'),
            ('expiry_dates', '📅 Certification Expiry'),
            ('job_position_name', '💼 Desired Position'),
            ('educational_requirements', '🎓 Education Requirements'),
            ('experience_requirement', '⏱️ Experience Required'),
            ('age_requirement', '👤 Age Requirement'),
            ('responsibilities.1', '📋 Additional Responsibilities'),
            ('skills_required', '🔧 Skills Required'),
            ('matched_score', '🎯 Matched Score')
        ]

        for col_name, emoji_label in dimensions:
            value = candidate.get(col_name, '')
            if value and str(value) != '' and str(value) != 'nan':
                # Truncate long values
                value_str = str(value)[:200]
                if len(str(value)) > 200:
                    value_str += "..."
                print(f"{emoji_label}: {value_str}")

        # Show text representation (what goes into embedding)
        print(f"\n📝 TEXT REPRESENTATION (for embedding):")
        print("-" * 80)
        print(candidate.get('text', 'N/A')[:500])
        if len(str(candidate.get('text', ''))) > 500:
            print("...")

    elif node_id.startswith('J'):
        # It's a COMPANY
        comp_idx = int(node_id[1:])

        if comp_idx >= len(companies_full):
            print(f"❌ Company {comp_idx} not found!")
            return

        company = companies_full.iloc[comp_idx]

        print(f"🔴 COMPANY #{comp_idx}")
        print("=" * 80)

        # Display company data
        print("\n📊 COMPANY INFORMATION:\n")

        company_fields = [
            ('company_id', '🆔 Company ID'),
            ('name', '🏢 Company Name'),
            ('description', '📄 Description'),
            ('industries_list', '🏭 Industries'),
            ('specialties_list', '⚙️ Specialties'),
            ('employee_count', '👥 Employee Count'),
            ('follower_count', '📈 Followers'),
            ('city', '📍 City'),
            ('state', '📍 State'),
            ('country', '🌍 Country'),
            ('posted_job_titles', '💼 Posted Jobs'),
            ('posted_descriptions', '📝 Job Descriptions'),
            ('required_skills', '🔧 Required Skills'),
            ('experience_levels', '⏱️ Experience Levels'),
            ('work_types', '💻 Work Types')
        ]

        for col_name, emoji_label in company_fields:
            value = company.get(col_name, '')
            if value and str(value) != '' and str(value) != 'nan':
                value_str = str(value)[:200]
                if len(str(value)) > 200:
                    value_str += "..."
                print(f"{emoji_label}: {value_str}")

        # Show text representation
        print(f"\n📝 TEXT REPRESENTATION (for embedding):")
        print("-" * 80)
        print(company.get('text', 'N/A')[:500])
        if len(str(company.get('text', ''))) > 500:
            print("...")

    else:
        print(f"❌ Invalid node ID: {node_id}")
        print("   Use format: 'C76' for candidates or 'J1234' for companies")

    print("\n" + "=" * 80 + "\n")


# ============================================================================
# DISPLAY NODE + ALL CONNECTIONS
# ============================================================================

def display_node_with_connections(node_id, top_k=10):
    """
    Display node data + all its connections.
    """

    # Display the node itself
    display_node_data(node_id)

    # If candidate, show their top matches
    if node_id.startswith('C'):
        cand_idx = int(node_id[1:])

        print(f"🎯 TOP {top_k} COMPANY MATCHES FOR CANDIDATE #{cand_idx}:")
        print("=" * 80)

        matches = find_top_matches(cand_idx, top_k=top_k)

        print(f"\n{'Rank':<6} {'Score':<8} {'Company':<35} {'Required Skills'}")
        print("-" * 80)

        for rank, (comp_idx, score) in enumerate(matches, 1):
            company = companies_full.iloc[comp_idx]
            name = company.get('name', 'N/A')[:33]
            skills = company.get('required_skills', 'N/A')[:40]
            print(f"{rank:<6} {score:.4f}   {name:<35} {skills}")

        print("\n" + "=" * 80)

        # Ask if user wants to see details
        print(f"\n💡 To see full details of a company, use:")
        print(f"   display_node_data('J{matches[0][0]}')")


# ============================================================================
# BATCH DISPLAY - Multiple nodes at once
# ============================================================================

def display_multiple_nodes(node_ids):
    """
    Display data for multiple nodes.

    Args:
        node_ids: list of str like ["C76", "J1234", "C80"]
    """
    for node_id in node_ids:
        display_node_data(node_id)
        print("\n")


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

# Example 1: Display Candidate 76
display_node_data('C76')

# Example 2: Display Candidate 76 + their top matches
display_node_with_connections('C76', top_k=10)

# Example 3: Display a specific company
display_node_data('J1234')

# Example 4: Display multiple nodes
display_multiple_nodes(['C76', 'C74', 'C23'])

# Example 5: Interactive selection
print("🔍 Enter node ID to display (e.g., 'C76' or 'J1234'):")
node_input = input().strip()
display_node_with_connections(node_input)

🟢 CANDIDATE #76

📊 ALL 35 DIMENSIONS:

🎯 Career Objective: Seeking Roles In Machine Learning, Natural Language Processing, Deep Learning. Previously worked as a Data Processing intern. Having good knowledge in using Python Libraries and different data process...
💻 Skills: ['Machine Learning', 'Natural Language Processing', 'Deep Learning', 'Business Analysis', 'Python', 'Numpy', 'Pandas', 'Scikit Learn', 'Tensorflow', 'Keras', 'PySpark']
🎓 Education Institution: ['Sl Thala Model college']
📜 Degrees: ['B.Tech']
📅 Graduation Years: ['2020']
⭐ Academic Results: ['N/A']
📊 Result Types: ['N/A']
📚 Major Field: ['N/A']
🏢 Companies Worked: ['SaiKo Computation']
🔗 Company URLs: [None]
📅 Start Dates: ['May 2019']
📅 End Dates: ['August 2019']
👔 Positions Held: ['Data Processing Junior Intern']
📍 Work Locations: ['N/A']
📋 Responsibilities: Machine Learning Leadership
Cross-Functional Collaboration
Strategy Development
ML/NLP Infrastructure
Prototype Transformation
ML System Design
Algorithm Resear